In [ ]:
import cv2
import imutils
import numpy as np
from configuration import *
import warnings
total_pos_image=1
total_neg_image=1

In [4]:
'''
Name of function: augmentation
Input: source image and rotate choice
Output: rotated image
Working: uses cv2.rotate
References:https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_geometric_transformations/py_geometric_transformations.html
'''
def augmentation(src, wish=0): ## function to rotate images
    if wish == 0:
        # Rotate 90
        src = cv2.rotate(src, rotateCode=cv2.ROTATE_90_CLOCKWISE)
    if wish == 1:
        # Rotate 90 and flip horizontally
        src = cv2.rotate(src, rotateCode=cv2.ROTATE_90_CLOCKWISE)
        src = cv2.flip(src, flipCode=1)
    if wish == 2:
        # Rotate 180
        src = cv2.rotate(src, rotateCode=cv2.ROTATE_180)
    if wish == 3:
        # Rotate 180 and flip horizontally
        src = cv2.rotate(src, rotateCode=cv2.ROTATE_180)
        src = cv2.flip(src, flipCode=1)
    if wish == 4:
        # Rotate 90 counter-clockwise
        src = cv2.rotate(src, rotateCode=cv2.ROTATE_90_COUNTERCLOCKWISE)
    if wish == 5:
        # Rotate 90 counter-clockwise and flip horizontally
        src = cv2.rotate(src, rotateCode=cv2.ROTATE_90_COUNTERCLOCKWISE)
        src = cv2.flip(src, flipCode=1)
    return src
'''------------------------
Name of function:
Input:
Output:
Working:
References:
------------------------'''
def increase_brightness(image, gamma=1.0):
    invGamma = 1.0 / gamma # increase brightness of the image.
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

'''------------------------
Name of function:
Input:
Output:
Working:
References:
------------------------'''
def rotate_angle(image,angle):
    return imutils.rotate_bound(image, angle)

'''------------------------
Name of function:
Input:
Output:
Working:
References:
------------------------'''
def crop_pos_image(image,image_name,x1,x2,y1,y2,pos_path):
    global total_pos_image
    try:
        img = image[x1:x2, y1:y2]
        if img.shape[0]>0 and img.shape[1]>0:
            file_path_pos = pos_path + str(total_pos_image)+".png"
            total_pos_image += 1
            file_path_pos_mirror=pos_path + str(total_pos_image)+".png"
            total_pos_image += 1
            file_path_pos_light = pos_path + str(total_pos_image) + ".png"
            total_pos_image += 1
            file_path_pos_light2 = pos_path + str(total_pos_image) + ".png"
            total_pos_image += 1
            cv2.imwrite(file_path_pos, img)
            cv2.imwrite(file_path_pos_mirror, augmentation(augmentation(img, 5), 4))
            cv2.imwrite(file_path_pos_light,increase_brightness(img,3))
            cv2.imwrite(file_path_pos_light2, increase_brightness(augmentation(augmentation(img, 5), 4), 3))
    except Exception as err:
        ERR=1
'''------------------------
Name of function:
Input:
Output:
Working:
References:
------------------------'''
def crop_neg_image(image,image_name,x1,x2,y1,y2,neg_path):
    global total_neg_image
    first_start = 0
    last_end = 0
    read_data = image
    try:
        height = read_data.shape[0]
        width = read_data.shape[1]
        window_width = [1, 1, 1, 1, 1, 1]
        window_height = [1, 1, 1, 1, 1, 1]
        image_array=[]
        first_start = int(y1 / 100)
        last_end = int((width-y2)/100)
        k = 0
        for i in range(first_start):
            window_width[i] = 100 * i
            k += 1
        if last_end == 3:
            window_width[k] = x2
            window_width[k + 1] = x2 + 100
            window_width[k + 2] = x2 + 200
        if last_end == 2:
            window_width[k] = x2
            window_width[k + 1] = x2 + 100
        if last_end == 1:
            window_width[k] = x2
        for i in range(len(window_width)):
            if window_width[i]==1:
                break
            else:
                file_path=neg_path + str(total_neg_image) + ".png"
                total_neg_image += 1
                file_path2=neg_path + str(total_neg_image) + ".png"
                total_neg_image += 1
                image2=image[x1:x2,window_width[i]:window_width[i]+100]
                if(image2.shape[0]==200 and image2.shape[1]==100):
                    cv2.imwrite(file_path,image2)
                    cv2.imwrite(file_path2, augmentation(augmentation(image2, 5), 4))
                else:
                    warnings.warn("Neg image is not consistent",Warning)
    except:
        warnings.warn("Size Inconsistency Error",Warning)
'''------------------------
Name of function:
Input:
Output:
Working:
References:
------------------------'''
def parse_image(path_idl,path_image,path_pos,path_neg):
    k=0
    print("Hello")
    for i in range(len(path_idl)):
        print(path_idl[i])
        with open(path_idl[i],"r") as ins:
            for line in ins:
                idl_line = line.split(":")
                file_name = idl_line[0].replace('"', "")
                main_img_path=path_image[i]+file_name
                image = cv2.imread(main_img_path, 0)
                image_coordinates=idl_line[1]
                if (image_coordinates[1] == '('):
                    image_coordinates = image_coordinates.replace("(", "")
                    image_coordinates = image_coordinates.replace(")", "")
                    image_coordinates = image_coordinates.split(",")
                    y1, x1, y2, x2 = int(image_coordinates[0]), int(image_coordinates[1]), int(image_coordinates[2]), int(image_coordinates[3])
                    p4 = (x1 - x2)
                    p5= (x2 - x1)
                    p2=(y1-y2)
                    p3=(y2-y1)
                    if(p2>0):
                        y1+=image_width-p2
                    elif(p3>0):
                        y1-=image_width-p3
                    if p4>0:
                        x1+=image_height-p4
                    elif p5 > 0:
                        x1 -= image_height-p5
                    if (abs(x1 - x2) == image_height):
                        k += 1
                    ## do image cropping here for positive instances
                    if(abs(x1-x2)==200):
                        crop_pos_image(image,file_name,x1,x2,y1,y2,path_pos)
                        crop_neg_image(image,file_name,x1,x2,y1,y2,path_neg)
                    #-----------------------
                for p1 in range(2, 6):
                    try:
                        image_coordinates = (idl_line[p1].replace("-1, ", ""))
                        image_coordinates = (image_coordinates.replace("-1;", ""))
                        if (image_coordinates[0] == '('):
                            image_coordinates = image_coordinates.replace("(", "")
                            image_coordinates = image_coordinates.replace(")", "")
                            image_coordinates = image_coordinates.split(",")
                            y1, x1, y2, x2 = int(image_coordinates[0]), int(image_coordinates[1]), int(image_coordinates[2]), int(image_coordinates[3])
                            p4 = (x1 - x2)
                            p5 = (x2 - x1)
                            p2 = (y1 - y2)
                            p3 = (y2 - y1)
                            if (p2 > 0):
                                y1 += image_width - p2
                            elif (p3 > 0):
                                y1 -= image_width - p3
                            if p4 > 0:
                                x1 += image_height - p4
                            elif p5 > 0:
                                x1 -= image_height - p5
                            if(abs(x1 - x2)==image_height):
                                k+=1
                            ## do image cropping here for positive instances
                            if(abs(x1-x2)==200):
                                crop_pos_image(image, file_name, x1, x2, y1, y2, path_pos)
                                crop_neg_image(image, file_name, x1, x2, y1, y2,path_neg)
                            ##-----------------------
                    except:
                        err=1

    return total_pos_image,total_neg_image


#--------------------------------------test-----------------------------------------
size_pos,size_neg=parse_image(path_idl_files,path_image_folders,total_pos_im_path,total_neg_im_path)
#-------------------------------------end of test-----------------------------------

Hello
data\train-210\train-210.idl


C:\Users\kankan\Anaconda3\lib\site-packages\ipykernel_launcher.py:126: Warning: Neg image is not consistent


data\train-400\train-400.idl


C:\Users\kankan\Anaconda3\lib\site-packages\ipykernel_launcher.py:128: Warning: Size Inconsistency Error
